In [1]:
class Event2:
    def __init__(self, timestamp, description):
        self.timestamp = timestamp
        self.description = description
        self.izquierda = None
        self.derecha = None
        self.altura = 1
        self.padre = None

class Arbre:
    def __init__(self):
        self.raiz = None

    def altura(self, nodo):
        if nodo is None:
            return 0
        return nodo.altura

    def factor_equilibrio(self, nodo):
        if nodo is None:
            return 0
        return self.altura(nodo.izquierda) - self.altura(nodo.derecha)

    def rotacion_derecha(self, z):
        y = z.izquierda
        T2 = y.derecha
        padre_z = z.padre

        y.derecha = z
        z.izquierda = T2
        z.padre = y

        if T2:
            T2.padre = z

        if z == self.raiz:
            self.raiz = y
        elif padre_z:
            if z == padre_z.izquierda:
                padre_z.izquierda = y
            else:
                padre_z.derecha = y

        y.padre = padre_z

        z.altura = 1 + max(self.altura(z.izquierda), self.altura(z.derecha))
        y.altura = 1 + max(self.altura(y.izquierda), self.altura(y.derecha))

        return y

    def rotacion_izquierda(self, z):
        y = z.derecha
        T2 = y.izquierda
        padre_z = z.padre

        y.izquierda = z
        z.derecha = T2
        z.padre = y

        if T2:
            T2.padre = z

        if z == self.raiz:
            self.raiz = y
        elif padre_z:
            if z == padre_z.izquierda:
                padre_z.izquierda = y
            else:
                padre_z.derecha = y

        y.padre = padre_z

        z.altura = 1 + max(self.altura(z.izquierda), self.altura(z.derecha))
        y.altura = 1 + max(self.altura(y.izquierda), self.altura(y.derecha))

        return y

    def preorden(self, raiz):
        if raiz:
            print(raiz.timestamp, end=" ")
            self.preorden(raiz.izquierda)
            self.preorden(raiz.derecha)

    def preorden_arbol(self):
        self.preorden(self.raiz)

    def insertar(self, raiz, timestamp, description, padre=None):
        if not raiz:
            nuevo_nodo = Event2(timestamp, description)
            nuevo_nodo.padre = padre
            return nuevo_nodo

        if timestamp <= raiz.timestamp:
            raiz.izquierda = self.insertar(raiz.izquierda, timestamp, description, raiz)
        else:
            raiz.derecha = self.insertar(raiz.derecha, timestamp, description, raiz)

        raiz.altura = 1 + max(self.altura(raiz.izquierda), self.altura(raiz.derecha))
        equilibrio = self.factor_equilibrio(raiz)

        if equilibrio > 1 and timestamp < raiz.timestamp:
            return self.rotacion_derecha(raiz)

        if equilibrio < -1 and timestamp > raiz.timestamp:
            return self.rotacion_izquierda(raiz)

        if equilibrio > 1 and timestamp > raiz.izquierda.timestamp:
            raiz.izquierda = self.rotacion_izquierda(raiz.izquierda)
            return self.rotacion_derecha(raiz)

        if equilibrio < -1 and timestamp < raiz.derecha.timestamp:
            raiz.derecha = self.rotacion_derecha(raiz.derecha)
            return self.rotacion_izquierda(raiz)

        return raiz

    def insertar_clave(self, timestamp, description):
        self.raiz = self.insertar(self.raiz, timestamp, description)
#/////////////////////////////////////////////////////////////////////////////////
#/////////////////////////////////////////////////////////////////////////////////


In [2]:
# Función externa
def add_event(timestamp, description, tree):
    tree.insertar_clave(timestamp, description)

# CAMBIOOO
def get_next_event(current_time, tree):
    result = None
    def buscar_proximo(nodo):
        nonlocal result
        if nodo is None:
            return
        if nodo.timestamp > current_time:
            if result is None or nodo.timestamp < result.timestamp:
                result = nodo
            buscar_proximo(nodo.izquierda)
        buscar_proximo(nodo.derecha)
    buscar_proximo(tree.raiz)
    return result

# CAMBIOOO
def get_events_in_range(start_time, end_time, tree):
    result = []
    def buscar_en_rango(nodo):
        if nodo is None:
            return
        if start_time <= nodo.timestamp <= end_time:
            buscar_en_rango(nodo.izquierda)
            result.append(nodo)
            buscar_en_rango(nodo.derecha)
        elif nodo.timestamp < start_time:
            buscar_en_rango(nodo.derecha)
        else:
            buscar_en_rango(nodo.izquierda)
    buscar_en_rango(tree.raiz)
    return result


In [3]:
# Crear el árbol AVL
scheduler_tree = Arbre()  # O RedBlackTree si tienes otra clase

# Agregar eventos
add_event(1000, "Reunión Equipo A", scheduler_tree)
add_event(1200, "Almuerzo", scheduler_tree)
add_event(900, "Llamada Cliente X", scheduler_tree)
add_event(1030, "Revisión Código", scheduler_tree)
add_event(1500, "Presentación Proyecto", scheduler_tree)

# Buscar próximo evento después de cierta hora
now = 1015
next_event = get_next_event(now, scheduler_tree)
if next_event:
    print(f"Próximo evento después de {now}: {next_event.description} a las {next_event.timestamp}")
else:
    print("No hay eventos próximos.")

# Buscar eventos en rango 

range_start = 950
range_end = 1250
events = get_events_in_range(range_start, range_end, scheduler_tree)
print(f"Eventos entre {range_start} y {range_end}:")
for e in events:
    print(f"- {e.description} a las {e.timestamp}")


Próximo evento después de 1015: Revisión Código a las 1030
Eventos entre 950 y 1250:
- Reunión Equipo A a las 1000
- Revisión Código a las 1030
- Almuerzo a las 1200


# Trying...